In [2274]:
# !pip install pandas
# !pip install skikit-learn
# !pip install --upgrade scikit-learn
# !pip install numpy

In [2275]:
import pandas as pd
import numpy as np

## Links and GET functions

In [2340]:
key = '41b145a848f4bd67'
def game_id_url_func(num_of_ahead_games, num_of_past_games, current_team_id=''):
  return f'https://lscluster.hockeytech.com/feed/?feed=modulekit&key={key}&view=scorebar&client_code=whl&numberofdaysahead={num_of_ahead_games}&numberofdaysback={num_of_past_games}&season_id=&team_id={current_team_id}&lang_code=en&fmt=json'

def game_stats_url_func(game_id):
  return f'https://lscluster.hockeytech.com/feed/?feed=gc&key={key}&game_id={game_id}&client_code=whl&tab=clock&lang_code=en&fmt=json'

def get_game_ids(url):
  headers = {
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/114.0.0.0 Safari/537.36",
        "Accept-Encoding": "gzip, deflate, br"  # Request gzip compression
  }

  try:
      response = requests.get(url, headers=headers)

      # Check if the request was successful
      if response.status_code == 200:

          return response.json()['SiteKit']['Scorebar']
          #pprint(response)
      else:
          print(f"Failed to retrieve the page. Status code: {response.status_code}")
  except Exception as e:
      print(f"An error occurred: {e}")


'''
Expects a game_id, which is used to access the Hockey Tech API to get the stats of a game, returned in a JSON format
'''
def get_game_stats(game_id):
  game_stats_url = game_stats_url_func(game_id)
  headers = {
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/114.0.0.0 Safari/537.36",
        "Accept-Encoding": "gzip, deflate, br"  # Request gzip compression
  }
  
  try:
      response = requests.get(game_stats_url, headers=headers)
  
      # Check if the request was successful
      if response.status_code == 200:
  
          return response.json()
          pprint(response)
      else:
          print(f"Failed to retrieve the page. Status code: {response.status_code}")
  except Exception as e:
      print(f"An error occurred: {e}")

## Import the Dataset

In [2470]:
dataset = pd.read_csv('All_teams_WHL_stats.csv')
target_team = 'Moose Jaw Warriors'
team_id = {'Spokane Chiefs': '215', 'Seattle Thunderbirds': '214', 'Portland Winterhawks': '208', 'Everett Silvertips': '226', 'Tri-City Americans': '217', 'Kamloops Blazers': '203', 'Kelowna Rockets': '204', 'Prince George Cougars': '210', 'Brandon Wheat Kings': '201', 'Swift Current Broncos': '216', 'Vancouver Giants': '223', 'Victoria Royals': '227', 'Medicine Hat Tigers': '206', 'Edmonton Oil Kings': '228', 'Moose Jaw Warriors': '207', 'Regina Pats': '212', 'Saskatoon Blades': '213', 'Prince Albert Raiders': '209', 'Calgary Hitmen': '202', 'Lethbridge Hurricanes': '205', 'Red Deer Rebels': '211', 'Wenatchee Wild': '222'}
k = 5 # trained on the last k games

In [2472]:
dataset = dataset.sort_values(by="Game_ID")
dataset.reset_index(drop=True, inplace=True)
dataset = dataset.drop(dataset.index[-1])
display(dataset)

,Unnamed: 0,Game_ID,Home_Name,Away_Name,Home_Goals,Away_Goals,Home_PP%,Away_PP%,Home_SOG,Away_SOG,Home_FOW%,Away_FOW%
0,5302,1014692,Prince George Cougars,Calgary Hitmen,7,1,0.285714,0.250000,34,17,0.433333,0.566667
1,5303,1014699,Kamloops Blazers,Calgary Hitmen,2,4,0.000000,0.000000,32,56,0.492958,0.507042
2,5304,1014708,Kelowna Rockets,Calgary Hitmen,3,4,0.166667,0.000000,34,26,0.461538,0.538462
3,5305,1014720,Calgary Hitmen,Lethbridge Hurricanes,1,3,0.000000,0.285714,31,37,0.500000,0.500000
4,5306,1014735,Red Deer Rebels,Calgary Hitmen,4,2,0.000000,0.000000,31,26,0.576271,0.423729
...,...,...,...,...,...,...,...,...,...,...,...,...
5592,4465,1021541,Edmonton Oil Kings,Prince Albert Raiders,2,5,0.333333,0.500000,34,33,0.586207,0.413793
5593,2449,1021542,Kelowna Rockets,Prince George Cougars,3,5,0.000000,1.000000,20,34,0.464286,0.535714
5594,499,1021543,Portland Winterhawks,Spokane Chiefs,6,4,0.333333,0.500000,47,30,0.677419,0.322581
5595,3550,1021544,Red Deer Rebels,Swift Current Broncos,4,5,0.250000,0.250000,33,35,0.514706,0.485294


## Update the dataset with recent games

In [2586]:
# Get the most recent games, and check their game ID
last_game_id = get_game_ids(game_id_url_func(0, 1))
last_game_id = int(last_game_id[-1]["ID"])

# Get the latest game ID we have in the dataset
last_recorded_game_id = dataset.iloc[-1]['Game_ID']

# Get the difference between the last game and the last recorded game.
# this will give us how many games off our dataset is
num_of_game_to_update = last_game_id - last_recorded_game_id
print(f'We need to update {num_of_game_to_update} games in our dataset')

We need to update 0 games in our dataset


In [2588]:
update_games = get_game_ids(game_id_url_func(0, num_of_game_to_update))
print(f'Number of games fetched {len(update_games)}')
# We get a lot of games because the API calls num_of_game_to_update back, and multiple games are played on the same day

Number of games fetched 0


In [2590]:
update_games = get_game_ids(game_id_url_func(0, num_of_game_to_update))


for game in update_games:
  # Because we got more games than we needed, we ignore the games we already have
  if int(game['ID']) <= last_recorded_game_id:
    continue

  # Get the game stats for each game
  stats = get_game_stats(game["ID"])

  print(stats)
  # Home and visitor team names
  visitor = stats['GC']['Clock']['visiting_team']['name']
  home = stats['GC']['Clock']['home_team']['name']

  # Number of goals
  home_goals = int(stats['GC']['Clock']['home_goal_count'])
  visitor_goals = int(stats['GC']['Clock']['visiting_goal_count'])

  # Calculate Power Play %
  home_ppp_total = float(stats['GC']['Clock']['power_play']['total']['home'])
  visitor_ppp_total = float(stats['GC']['Clock']['power_play']['total']['visiting'])
  # Avoid division by zero for home_ppp
  if home_ppp_total != 0:
      home_ppp = float(stats['GC']['Clock']['power_play']['goals']['home']) / home_ppp_total
  else:
      home_ppp = 0  # Default to 0 if no power plays
  # Avoid division by zero for visitor_ppp
  if visitor_ppp_total != 0:
      visitor_ppp = float(stats['GC']['Clock']['power_play']['goals']['visiting']) / visitor_ppp_total
  else:
      visitor_ppp = 0  # Default to 0 if no power plays

  # Calculate Faceoff Win %
  home_fowp = float(stats['GC']['Clock']['fow']['home'])
  visitor_fowp = float(stats['GC']['Clock']['fow']['visiting'])
  fow_total = home_fowp + visitor_fowp
  if fow_total != 0:
      home_fowp /= fow_total
      visitor_fowp /= fow_total
  else:
      home_fowp, visitor_fowp = .5, .5

  # Shots on goal
  home_sog = sum(stats['GC']['Clock']['shots_on_goal']['home'].values())
  visitor_sog = sum(stats['GC']['Clock']['shots_on_goal']['visiting'].values())
  
  # Add data to the dataset
  dataset.loc[len(dataset)] = {
      "Game_ID": int(game['ID']),
      "Home_Name": home,
      "Away_Name": visitor,
      "Home_Goals": home_goals,
      "Away_Goals": visitor_goals,
      "Home_PP%": home_ppp,
      "Away_PP%": visitor_ppp,
      "Home_SOG": home_sog,
      "Away_SOG": visitor_sog,
      "Home_FOW%": home_fowp,
      "Away_FOW%": visitor_fowp
  }

In [2592]:
# dataset = dataset.drop(columns=['Unnamed: 0'])
display(dataset)

,Game_ID,Home_Name,Away_Name,Home_Goals,Away_Goals,Home_PP%,Away_PP%,Home_SOG,Away_SOG,Home_FOW%,Away_FOW%
0,1014692,Prince George Cougars,Calgary Hitmen,7,1,0.285714,0.250000,34,17,0.433333,0.566667
1,1014699,Kamloops Blazers,Calgary Hitmen,2,4,0.000000,0.000000,32,56,0.492958,0.507042
2,1014708,Kelowna Rockets,Calgary Hitmen,3,4,0.166667,0.000000,34,26,0.461538,0.538462
3,1014720,Calgary Hitmen,Lethbridge Hurricanes,1,3,0.000000,0.285714,31,37,0.500000,0.500000
4,1014735,Red Deer Rebels,Calgary Hitmen,4,2,0.000000,0.000000,31,26,0.576271,0.423729
...,...,...,...,...,...,...,...,...,...,...,...
5604,1021555,Victoria Royals,Prince George Cougars,1,2,0.000000,0.000000,29,34,0.510204,0.489796
5605,1021554,Spokane Chiefs,Wenatchee Wild,2,1,0.200000,0.200000,24,33,0.573770,0.426230
5606,1021553,Seattle Thunderbirds,Everett Silvertips,3,5,0.500000,0.500000,27,40,0.507246,0.492754
5607,1021554,Spokane Chiefs,Wenatchee Wild,2,1,0.200000,0.200000,24,33,0.573770,0.426230


## Create features and dependencies

In [2595]:
# Get default values for averages of goals, PP%, SOG, and FOW%
default_goals = (dataset['Home_Goals'].mean() + dataset['Away_Goals'].mean()) // 2
default_ppp = (dataset['Home_PP%'].mean() + dataset['Away_PP%'].mean()) / 2
default_sog = (dataset['Home_SOG'].mean() + dataset['Away_SOG'].mean()) // 2
default_fowp = (dataset['Home_FOW%'].mean() + dataset['Away_FOW%'].mean()) / 2

print(default_goals, default_ppp, default_sog, default_fowp)

3.0 0.2153101707032889 31.0 0.5


In [2533]:
def get_opponent_stats(dataset, queue, opponent_name):
  opponent_goals = 0
  opponent_ppp = 0
  opponent_fowp = 0
  opponent_sog = 0
  # Get the last k games averages
  # This creates a new dataset of only opponent games, and which are before the current game
  opponent_games = dataset[
    ((dataset['Home_Name'] == opponent_name) | (dataset['Away_Name'] == opponent_name)) & 
    (dataset['Game_ID'] < row['Game_ID'])
  ].tail(k)

  # If the opponent hasnt played k games yet, use default
  if len(opponent_games) < k:
    opponent_goals = default_goals
    opponent_ppp = default_ppp
    opponent_fowp = default_fowp
    opponent_sog = default_sog
  else:
    # Get the mean of the entire column that the opponent played
    opponent_goals = opponent_games['Home_Goals'].where(opponent_games['Home_Name'] == opponent_name, opponent_games['Away_Goals']).mean()
    opponent_ppp = opponent_games['Home_PP%'].where(opponent_games['Home_Name'] == opponent_name, opponent_games['Away_PP%']).mean()
    opponent_fowp = opponent_games['Home_FOW%'].where(opponent_games['Home_Name'] == opponent_name, opponent_games['Away_FOW%']).mean()
    opponent_sog = opponent_games['Home_SOG'].where(opponent_games['Home_Name'] == opponent_name, opponent_games['Away_SOG']).mean()

  return opponent_goals, opponent_ppp, opponent_fowp, opponent_sog

In [2535]:
from collections import deque
target_queue = deque(maxlen=k)
target_df = pd.DataFrame(columns = [
    "target_goals", 
    "opponent_goals", 
    "target_PP%", 
    "opponent_PP%", 
    "target_SOG", 
    "opponent_SOG",
    "target_FOW%",
    "opponent_FOW%",
    "Home/Away",
    "opponent_win_loss",
    "goals_diff",
    "ppp_diff",
    "sog_diff",
    "fowp_diff",
    "target_win"
])

target_goals = 0
target_ppp = 0
target_fowp = 0
target_sog = 0

opponent_win_loss = {}

# Iterate through the DataFrame to track the last 5 games
for index, row in dataset.iterrows():
  if row['Home_Name'] == target_team or row['Away_Name'] == target_team:

    side = 'Home' if row['Home_Name'] == target_team else 'Away'
    opponent_name = row['Away_Name'] if side == 'Home' else row['Home_Name']
    # now we are within the target team
    # If the queue is less than k, just add to queue
    target_goals += row[side + '_Goals']
    target_ppp += row[side + '_PP%']
    target_fowp += row[side + '_FOW%']
    target_sog += row[side + '_SOG']
   
    
    if len(target_queue) < k:
      target_queue.append(row.to_dict())
    else: #If queue is full, then we can do some popping

      opponent_goals, opponent_ppp, opponent_fowp, opponent_sog = get_opponent_stats(dataset, target_queue, opponent_name)

      target_win = 1 if row[side + '_Goals'] > row[('Away' if side == 'Home' else 'Home') + '_Goals'] else 0
      if opponent_name not in opponent_win_loss:
        opponent_win_loss[opponent_name] = 0.0
      
      # Update opponent win/loss balance
      if target_win:
        opponent_win_loss[opponent_name] -= 1.0
      else:
        opponent_win_loss[opponent_name] += 1.0
      
      #print(target_goals / k, target_ppp / k, target_fowp / k, target_sog / k)
      # print(opponent_goals, opponent_ppp, opponent_fowp, opponent_sog)
      # Calculate average stats for the target team
      avg_target_goals = target_goals / k
      avg_target_ppp = target_ppp / k
      avg_target_fowp = target_fowp / k
      avg_target_sog = target_sog / k

      # Calculate differences between the target team and the opponent
      goals_diff = avg_target_goals - opponent_goals
      ppp_diff = avg_target_ppp - opponent_ppp
      sog_diff = avg_target_sog - opponent_sog
      fowp_diff = avg_target_fowp - opponent_fowp

      # 1 is Home, 0 is Away
      target_df.loc[len(target_df)] = {
        "target_goals": avg_target_goals,
        "opponent_goals": opponent_goals,
        "target_PP%": avg_target_ppp,
        "opponent_PP%": opponent_ppp,
        "target_SOG": avg_target_sog,
        "opponent_SOG": opponent_sog,
        "target_FOW%": avg_target_fowp,
        "opponent_FOW%": opponent_fowp,
        "Home/Away": 1.0 if side == 'Home' else 0.0,
        "opponent_win_loss": opponent_win_loss[opponent_name],
        "goals_diff": goals_diff,          # New column for target vs opponent goals difference
        "ppp_diff": ppp_diff,            # New column for PP% difference
        "sog_diff": sog_diff,            # New column for SOG difference
        "fowp_diff": fowp_diff,
        "target_win": target_win
      }

      popped = target_queue.popleft()
      popped_side ='Home' if popped['Home_Name'] == target_team else 'Away'
      target_goals -= popped[popped_side + '_Goals']
      target_ppp -= popped[popped_side + '_PP%']
      target_fowp -= popped[popped_side + '_FOW%']
      target_sog -= popped[popped_side + '_SOG']

      target_queue.append(row.to_dict())
        
        


In [2536]:
display(target_df)

,target_goals,opponent_goals,target_PP%,opponent_PP%,target_SOG,opponent_SOG,target_FOW%,opponent_FOW%,Home/Away,opponent_win_loss,goals_diff,ppp_diff,sog_diff,fowp_diff,target_win
0,5.8,3.2,0.503333,0.275000,41.0,28.6,0.576986,0.522733,0.0,-1.0,2.6,0.228333,12.4,0.054254,1
1,6.4,5.0,0.436667,0.340000,43.0,31.0,0.591577,0.471331,1.0,-1.0,1.4,0.096667,12.0,0.120246,1
2,7.2,4.8,0.503333,0.256667,39.8,33.4,0.584020,0.477526,1.0,-1.0,2.4,0.246667,6.4,0.106494,1
3,7.0,3.4,0.463333,0.218571,40.6,25.6,0.610427,0.523317,0.0,1.0,3.6,0.244762,15.0,0.087110,0
4,7.2,4.2,0.396667,0.285238,42.6,28.0,0.583761,0.531831,1.0,0.0,3.0,0.111429,14.6,0.051930,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
520,4.0,3.6,0.183333,0.213333,34.4,31.8,0.509827,0.544709,0.0,4.0,0.4,-0.030000,2.6,-0.034882,0
521,3.2,4.8,0.233333,0.333333,33.0,36.8,0.476540,0.509805,1.0,0.0,-1.6,-0.100000,-3.8,-0.033266,0
522,2.6,2.8,0.216667,0.223333,31.2,30.2,0.464727,0.517384,0.0,-8.0,-0.2,-0.006667,1.0,-0.052657,1
523,3.6,4.8,0.283333,0.250000,33.2,34.0,0.468837,0.526912,0.0,-1.0,-1.2,0.033333,-0.8,-0.058075,1


# Machine Learning Classification

## Split the dataset

In [2541]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, accuracy_score
X = target_df.iloc[:, :-1]
y = target_df.iloc[:, -1]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 0)

## Scale the features

In [2544]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()

X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

## Apply Classification Model

### Random Forest Classification

In [2548]:
from sklearn.ensemble import RandomForestClassifier
# This is an ensemble. (A bunch of smaller classification trees)
# Again, this has a random state
# Because our data set is so simple we will just to based on 40 trees
#   Even with a larger amount of esimators to results remain the same or worse
rfc_classifier = RandomForestClassifier(n_estimators=100, criterion='entropy', random_state=0)
rfc_classifier.fit(X_train, y_train)
y_pred = rfc_classifier.predict(X_test)
cm = confusion_matrix(y_test, y_pred)
print(cm)

rfc_accuracy = accuracy_score(y_test, y_pred)

[[34 16]
 [17 38]]


### Naive Bayes

In [2551]:
from sklearn.naive_bayes import GaussianNB
# New library from sklearn
#   Lookup sklearn Naive Bayes
# This is non-linear and will consider probability
nb_classifier = GaussianNB()
nb_classifier.fit(X_train, y_train)

y_pred = nb_classifier.predict(X_test)
cm = confusion_matrix(y_test, y_pred)
print(cm)

nb_accuracy = accuracy_score(y_test, y_pred)

[[34 16]
 [17 38]]


### Logistic Regression

In [2554]:
from sklearn.linear_model import LogisticRegression

lg_classifier = LogisticRegression(random_state=0)
lg_classifier.fit(X_train, y_train)

y_pred = lg_classifier.predict(X_test)
cm = confusion_matrix(y_test, y_pred)
print(cm)

lg_accuracy = accuracy_score(y_test, y_pred)

[[34 16]
 [13 42]]


### Support Vector Machine

In [2557]:
from sklearn.svm import SVC
# We are using the radial basis function
# This is a non-linear function
svc_classifier = SVC(kernel='rbf', random_state=0)
svc_classifier.fit(X_train, y_train)

y_pred = svc_classifier.predict(X_test)
cm = confusion_matrix(y_test, y_pred)
print(cm)

svc_accuracy = accuracy_score(y_test, y_pred)

[[32 18]
 [21 34]]


In [2559]:
count_target_win = target_df['target_win'].value_counts()[1]
print(f"Number of times target_win is 1: {count_target_win}")

count_target_loss = target_df['target_win'].value_counts()[0]
print(f"Number of times target_win is 0: {count_target_loss}")

target_total = count_target_loss + count_target_win
print(f"Win %: {count_target_win / target_total}")
print(f"Loss %: {count_target_loss / target_total}")

# print(len(dataset['Home_Name'] == target_team))
print((dataset['Away_Name'] == target_team).sum())
print((dataset['Home_Name'] == target_team).sum())


Number of times target_win is 1: 264
Number of times target_win is 0: 261
Win %: 0.5028571428571429
Loss %: 0.49714285714285716
266
264


# Predict Future Games

In [2563]:
import requests

In [2565]:
def get_last_k_stats(game_ids, metrics, team_name):
  for i in game_ids:
    side = 'home' if i['HomeLongName'] == team_name else 'visiting'

    print(team_name + " " + i["ID"])
    stats = get_game_stats(i['ID'])

    metrics["goals"] += int(stats['GC']['Clock'][side + '_goal_count'])


    ppp_total = float(stats['GC']['Clock']['power_play']['total'][side])
    if ppp_total != 0:
      metrics["ppp"] += float(stats['GC']['Clock']['power_play']['goals'][side]) / ppp_total
    else:
      metrics["ppp"] += 0  # Default to 0 if no power plays

    # Calculate the Face off win %
    fow = float(stats['GC']['Clock']['fow'][side])
    opponent_fow = float(stats['GC']['Clock']['fow']['home' if side == 'visiting' else 'visiting'])
    fow_total = fow + opponent_fow
    if fow != 0:
       metrics["fowp"] += (fow / fow_total)
    else:
       metrics["fowp"] += 0.5

    # Get the shots on goal
    metrics["sog"] += sum(stats['GC']['Clock']['shots_on_goal'][side].values())


In [2567]:
# Get the next game playing
future_game_response = get_game_ids(game_id_url_func(1, 0, team_id[target_team]))[0]
#print(future_game_response)

# Get the opponent for the future game
future_opponent_name = future_game_response['HomeLongName'] if future_game_response['HomeLongName'] != target_team else future_game_response['VisitorLongName']

# Get the past k games played. We will need those stats
target_games_response = get_game_ids(game_id_url_func(0, k, team_id[target_team]))
opponent_games_response = get_game_ids(game_id_url_func(0, k, team_id[future_opponent_name]))

target_metrics = {"goals": 0, "ppp": 0, "fowp": 0, "sog": 0}
opponent_metrics = {"goals": 0, "ppp": 0, "fowp": 0, "sog": 0}

# Get the stats of the last k games combines
get_last_k_stats(target_games_response, target_metrics, target_team)
get_last_k_stats(opponent_games_response, opponent_metrics, future_opponent_name)
print(target_metrics, opponent_metrics)
# divide by k to get the avg stats of the last k games
target_metrics = {key: value / k for key, value in target_metrics.items()}
opponent_metrics = {key: value / k for key, value in opponent_metrics.items()}
print(target_metrics, opponent_metrics)

Moose Jaw Warriors 1021511
Moose Jaw Warriors 1021519
Moose Jaw Warriors 1021532
Moose Jaw Warriors 1021540
Moose Jaw Warriors 1021552
Brandon Wheat Kings 1021496
Brandon Wheat Kings 1021530
Brandon Wheat Kings 1021538
Brandon Wheat Kings 1021540
Brandon Wheat Kings 1021546
{'goals': 15, 'ppp': 1.4999999999999998, 'fowp': 1.992532023986942, 'sog': 145} {'goals': 21, 'ppp': 1.5, 'fowp': 2.666508256528418, 'sog': 179}
{'goals': 3.0, 'ppp': 0.29999999999999993, 'fowp': 0.3985064047973884, 'sog': 29.0} {'goals': 4.2, 'ppp': 0.3, 'fowp': 0.5333016513056836, 'sog': 35.8}


## Now predict the values

In [2569]:
target_goals = target_metrics["goals"]
opponent_goals = opponent_metrics["goals"]
target_PPP = target_metrics["ppp"]
opponent_PPP = opponent_metrics["ppp"]
target_SOG = target_metrics["sog"]
opponent_SOG = opponent_metrics["sog"]
target_FOWP = target_metrics["fowp"]
opponent_FOWP = opponent_metrics["fowp"]
side = 1.0 if future_game_response['HomeLongName'] == target_team else 0.0
opponent_win_loss_curr = opponent_win_loss[future_opponent_name]
goals_diff = target_goals - opponent_goals
ppp_diff = target_PPP - opponent_PPP
sog_diff = target_SOG - opponent_SOG
fowp_diff = target_FOWP - opponent_FOWP

# Features to pass into the prediction
features = [
    [
        target_goals, opponent_goals, target_PPP, opponent_PPP,
        target_SOG, opponent_SOG, target_FOWP, opponent_FOWP,
        side, opponent_win_loss_curr, goals_diff, ppp_diff, sog_diff, fowp_diff
    ]
]

feature_names = ["target_goals", 
    "opponent_goals", 
    "target_PP%", 
    "opponent_PP%", 
    "target_SOG", 
    "opponent_SOG",
    "target_FOW%",
    "opponent_FOW%",
    "Home/Away",
    "opponent_win_loss",
    "goals_diff",
    "ppp_diff",
    "sog_diff",
    "fowp_diff"]


# Convert features to a DataFrame with the correct column names
features_df = pd.DataFrame(features, columns=feature_names)

# Now scale and predict
scaled_features = scaler.transform(features_df)

if rfc_accuracy >= nb_accuracy and rfc_accuracy >= lg_accuracy and rfc_accuracy >= svc_accuracy:
    best_classifier = rfc_classifier
    best_model_name = "Random Forest"
elif nb_accuracy >= rfc_accuracy and nb_accuracy >= lg_accuracy and nb_accuracy >= svc_accuracy:
    best_classifier = nb_classifier
    best_model_name = "Naive Bayes"
elif lg_accuracy >= rfc_accuracy and lg_accuracy >= nb_accuracy and lg_accuracy >= svc_accuracy:
    best_classifier = lg_classifier
    best_model_name = "Logistic Regression"
else:
    best_classifier = svc_classifier
    best_model_name = "SVM"

print(f'Using {best_model_name} Classifier')
prediction = best_classifier.predict(scaled_features)

game_date = future_game_response['GameDate']
if prediction:
  print(f"The {target_team} is predicted to win their next game against {future_opponent_name} on {game_date}")
else:
  print(f"The {future_opponent_name} is predicted to win the next game against {target_team} on {game_date}")

Using Logistic Regression Classifier
The Brandon Wheat Kings is predicted to win the next game against Moose Jaw Warriors on Sat, Dec 28
